In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import gc

# BOKEH
import bokeh.plotting as bk
import bokeh.models as bkmod
import bokeh.layouts as bklay

bk.output_notebook()

# Local imports
#------------------------------------------------
import WireDAQ.PandasPlus           # Make sure this import is after pandas
import WireDAQ.Constants as cst
import WireDAQ.NXCALS as nx
import WireDAQ.Parser as parser
import WireDAQ.Efficiency as eff


# Creating NXCALS variable containers
#------------------------------------------------
wires     = {'B1': [nx.NXCALSWire(loc = loc) for loc in ['L1B1','L5B1']],
             'B2': [nx.NXCALSWire(loc = loc) for loc in ['R1B2','R5B2']]}
beams     = [nx.NXCALSBeam(name) for name in ['B1','B2']]
LHC       = nx.NXCALSLHC()
b_slots   = np.arange(3564)
#------------------------------------------------


# Setting default values
#------------------------------------------------
_default_fig_width  = 2000
_default_fig_height = 400
_default_fig_pad    = 100

_default_device = 'DBLM'

_default_path     = '/eos/project/l/lhc-lumimod/LuminosityFollowUp/2023/'
_default_out      = '/eos/user/p/phbelang/www/Monitoring_BBCW/DBLM'




In [ ]:
FILL = 8850

# Fixing data path
data_path   = _default_path
device      = _default_device
raw_data    = data_path + '/rawdata/'
device_data = data_path + f'/efficiency_data/{device}/'



# Finding filling pattern
#-------------------------------------------------
bb_df_b1,bb_df_b2 = parser.fill_filling_pattern(fill=FILL,data_path= raw_data,n_LR = 21)
#-------------------------------------------------



# Declaring master bin times
#-------------------------------------------------
dt = 60
unix_s,unix_e = parser.fill_unix_times(FILL,data_path=raw_data)
unix_bins     = np.arange(unix_s,unix_e,dt/1e-9)
#-------------------------------------------------

# Import efficiency
#-------------------------------------------------
df_eff = parser.from_parquet2bin(file= device_data + f'/FILL{FILL}.parquet',bins=unix_bins)
#-------------------------------------------------

df_eff = parser.make_timestamps(df_eff)

In [ ]:
1/df_eff.set_index('Timestamp')[f'{beam.name}:eta'] - 1

In [ ]:
def make_efficiency_source(FILL,database,bb_df_list=None,slider_dt = 120,left=None,right=None):
    # Creating shared source for plots
    #=====================================================================
    source = {}
    data_bb = {}
    data_I  = {}
    for beam,bb_df,color in zip(beams,bb_df_list,['royalblue','firebrick']):
        
        
        # Extracting efficiency
        observable   = f'{beam.name}:eta'
        _times,_data = database.set_index('Timestamp')[observable].dropna().to_2D()
        _times       = _times.to_list()

        # Computing bunch-by-bunch data
        observable        = f'{beam.name}:eta'
        _data_bb          = database.set_index('Timestamp')[observable].dropna()
        _data_bb          = 1/_data_bb - 1
        data_bb[beam.name] = _data_bb.apply(lambda line: line[bb_df.index])

        # Computing average intensity
        _data_I           = database.set_index('Timestamp')[observable].dropna()
        data_I[beam.name] = _data_I.groupby(pd.Grouper(freq=f'{slider_dt}s')).mean().apply(lambda line: line[bb_df.index]/1e11)


        # Compiling data to source
        _source_df   = pd.DataFrame({   'Bunch'     :bb_df.index,
                                        'Intensity' :data_I[beam.name][left + (right-left)/2:][0],
                                        'data_l'    :data_bb[beam.name][left:][0],
                                        'data_avg'  :data_bb[beam.name][left:right].mean(),
                                        'data_r'    :data_bb[beam.name][right:][0],
                                        'Timestamp' :[_times       for idx in bb_df.index],
                                        'eta'       :[_data[:,idx] for idx in bb_df.index]})
        
        source[beam.name] = bkmod.ColumnDataSource(_source_df)
    #=====================================================================

    return source,data_I,data_bb


# New axis function
#=====================================
def new_axis(fig,axis_name,side='none'):
    fig.extra_y_ranges[axis_name] = bkmod.Range1d(0,1)
    _ax = bkmod.LinearAxis(y_range_name=axis_name)
    if side == 'none':
        pass
    else:
        fig.add_layout(_ax,side)

    return _ax,axis_name
#=====================================


class Windower():
    def __init__(self,_fig,left,right,colors = {'left':'coral','right':'mediumseagreen'},lw=pd.Timedelta(minutes=2),box_kwargs = {},bar_kwargs = {}):
        
        # Icon file
        self.icon     = Path('WireDAQ/Icons/slider_icon.png')

        # Creating new axis from 0-1 on the figure
        ax,axis_name = new_axis(_fig,axis_name='_slider')
        _fig.extra_y_ranges[axis_name] = bkmod.Range1d(0,1)
        bar_kwargs.update({'y_range_name':axis_name})

        # Metadata
        w_factor = 3
        self.metadata  = bkmod.ColumnDataSource({'left': [left],'right':[right],'lw':[lw] ,'yt':[2],'yb':[-1],'ym':[0.5]})

        # Range object
        self.range = bkmod.Range1d(left+lw/2,right-lw/2)


        # Add tool to figure
        self.RangeTool = bkmod.RangeTool(   x_range      = self.range,
                                            icon         = self.icon)
        
        box_kwargs.update({'level':'underlay'})
        self.RangeTool.overlay.update(**box_kwargs)
        #       visible      = False,
        #                                     fill_color   = None,
        #                                     fill_alpha   = 0,
        #                                     line_color   = None)
        _fig.add_tools(self.RangeTool)


        # Add visible slider itself
        self.renderer_l = _fig.rect(x="left"    , y="ym", width="lw", height="yt", source=self.metadata,line_color = colors['left'] ,fill_color=colors['left'] ,**bar_kwargs)
        self.renderer_r = _fig.rect(x="right"   , y="ym", width="lw", height="yt", source=self.metadata,line_color = colors['right'],fill_color=colors['right'],**bar_kwargs)


        # Muting for overlay tool
        self.renderer_l.muted = True
        self.renderer_r.muted = True
        self.renderer_l.muted_glyph = self.renderer_l.glyph.clone()
        self.renderer_r.muted_glyph = self.renderer_r.glyph.clone()


        # Default callback to link the range with rendered object
        #-------------------------
        self.callback_start_0 = bkmod.callbacks.CustomJS(args=dict(metadata = self.metadata), 
                                                                            code="""
                //=========================================================
                metadata.data['left'] = [cb_obj.start-metadata.data['lw'][0]/2];
                metadata.change.emit()
                //=========================================================""")

        self.callback_end_0 = bkmod.callbacks.CustomJS(args=dict(metadata = self.metadata), 
                                                                            code="""
                //=========================================================
                metadata.data['right'] = [cb_obj.end+metadata.data['lw'][0]/2];
                metadata.change.emit()
                //=========================================================""")

        self.callback_start = self.callback_start_0.clone()
        self.callback_end   = self.callback_end_0.clone()
        self.range.js_on_change('start',self.callback_start )
        self.range.js_on_change('end'  ,self.callback_end )
        #-------------------------
        
    def add_renderer(self,_fig,**kwargs):
        # Creating new axis from 0-1 on the figure
        _,axis_name = new_axis(_fig,axis_name='_slider_indicator')
        _fig.extra_y_ranges[axis_name] = bkmod.Range1d(0,1)
        kwargs.update({'y_range_name':axis_name})

        _rend = _fig.rect(x="x", y="ym", width="w", height="yt", source=self.metadata,**kwargs)
        # Muting for overlay tool
        _rend.muted = True
        if 'y_range_name' in kwargs.keys():
            kwargs.pop('y_range_name');
        _rend.muted_glyph.update(**kwargs)
        return _rend
    
    def update(self,**kwargs):
        if 'y_range_name' in kwargs.keys():
            kwargs.pop('y_range_name');
        self.renderer.glyph.update(**kwargs)
        self.renderer.muted_glyph.update(**kwargs)

    def add_start_callback(self,args,code):
        _args = dict(self.callback_start_0.args)
        _args.update(args)
        self.callback_start.update(args=_args, 
                                   code=self.callback_start_0.code + '\n' + code)

    def add_end_callback(self,args,code):
        _args = dict(self.callback_end_0.args)
        _args.update(args)
        self.callback_end.update(   args=_args, 
                                    code=self.callback_end_0.code + '\n' + code)
        

def make_bbbsignature_figure(source,beam,colors = {'left':'coral','right':'mediumseagreen','avg':'slateblue'}):

    # Creating Figure
    #=====================================
    fig = bk.figure(output_backend  = "webgl",
                    height          = _default_fig_height, 
                    width           = _default_fig_width,
                    title           = "Normalised losses", 
                    tools           = "box_zoom,pan,reset,save,hover",
                    active_drag     = "box_zoom",
                    toolbar_location= "right")

    # No grid
    fig.grid.visible = False
    
    # Saving tools to tags
    fig.tags = [{str(type(t)).split('.')[-1].split('\'')[0]:t for t in fig.tools}]
    # fig.tags[0]['BoxZoomTool'].update(dimensions = 'width')
    # fig.tags[0]['PanTool'].update(dimensions = 'width')
    fig.tags[0]['HoverTool'].update(tooltips = [(f'Beam', '$name'),('Bunch slot','$x{0}')])
    #=====================================


    

    c_left  = fig.circle(x='Bunch', y='data_l', color=colors['left'],name=f"{beam.name}",legend_label= 'Cursor 1',source=source[beam.name])
    c_avg   = fig.circle(x='Bunch', y='data_avg', color=colors['avg'],name=f"{beam.name}",legend_label= 'Average',source=source[beam.name]) 
    c_right = fig.circle(x='Bunch', y='data_r', color=colors['right'],name=f"{beam.name}",legend_label= 'Cursor 2',source=source[beam.name])

    # vbars.selection_glyph = bkmod.glyphs.VBar(line_color='black',fill_color=color)
    # vbars.nonselection_glyph = bkmod.glyphs.VBar(fill_color=color,fill_alpha=0.3,line_color=None)

    # Vertical line
    # l1 = bkmod.Span(location=0, dimension='height', line_color='black',line_alpha=0.5)
    # l2 = bkmod.Span(location=len(b_slots), dimension='height', line_color='black',line_alpha=0.5)
    # fig.renderers.extend([l1,l2])
    fig.xaxis.axis_label = "Bunch slot"
    fig.yaxis.axis_label = "Normalised losses: losses/(sigma*lumi)"
    # fig.yaxis.ticker     = [0,1]
    fig.x_range          = bkmod.Range1d(-10, len(b_slots)+10)
    # fig.yaxis.formatter  = bkmod.NumeralTickFormatter(format="0.0")
    fig.y_range          = bkmod.Range1d(0, 1.05*2)


    # Legend Options
    #=====================================
    fig.legend.location     = "top_left"
    fig.legend.click_policy = "hide"
    #=====================================legend_label= 'Cursor data',

    return fig


In [ ]:
main = __import__('002_make_HTML')

beam = beams[1]
color= 'firebrick'


# Making data source for most plots
#-------------------------------------
slider_dt     = dt
default_ts    = df_eff['Timestamp'].iloc[len(df_eff)//2]
left_ts       = default_ts - pd.Timedelta(minutes=5)
right_ts      = default_ts + pd.Timedelta(minutes=5)
source,data_I,data_bb = make_efficiency_source(FILL,df_eff,bb_df_list=[bb_df_b1,bb_df_b2],slider_dt=slider_dt,left=left_ts,right=right_ts)
#-------------------------------------

# Main plot
#-------------------------------------
BOKEH_FIGS = {}
BOKEH_FIGS[f'TOP'] = main.make_efficiency_figure(df_eff,source,beam,color)

# Add slider tool
#-------------------------------------
time_slider = Windower(BOKEH_FIGS[f'TOP'],
                       left     = left_ts,
                       right    = right_ts,
                       colors   = {'left':'tomato','right':'mediumseagreen'},
                       lw       = pd.Timedelta(minutes=0.5),
                       box_kwargs = dict(fill_color='black',fill_alpha=0.2,line_alpha=0),
                       bar_kwargs = dict(fill_alpha=0.5,line_width=2,line_alpha=0.5))

time_slider.add_start_callback(args = dict( sourceb1= source['B1'],
                                            sourceb2= source['B2'],
                                            ts_list = list(data_bb['B1'].index),
                                            bb_b1   = data_bb['B1'],
                                            bb_b2   = data_bb['B2']),
                                            code    =  """
    //====================================================
    //--------------------------------
    function addvector(a,b){
        return a.map((e,i) => e + b[i]);
    }

    function divvector(a,b){
        return a.map((e,i) => e/b);
    }

    function average2D(array) {
        return divvector(array.reduce((x,y) => addvector(x,y)),array.length);
    }
    //--------------------------------

    const _s     = cb_obj.start-metadata.data['lw'][0]/2;
    const _s_idx = ts_list.findIndex( x => Date.parse(new Date(x)) >= Date.parse(new Date(_s)));
    const _e     = cb_obj.end+metadata.data['lw'][0]/2;
    const _e_idx = ts_list.findIndex( x => Date.parse(new Date(x)) >= Date.parse(new Date(_e)));

    sourceb1.data.data_l = bb_b1[_s_idx];
    sourceb2.data.data_l = bb_b2[_s_idx];

    

    sourceb1.data.data_avg = average2D(bb_b1.slice(_s_idx,_e_idx));
    sourceb2.data.data_avg = average2D(bb_b2.slice(_s_idx,_e_idx));

    sourceb1.change.emit();
    sourceb2.change.emit();
    //====================================================""")

time_slider.add_end_callback(args = dict(   sourceb1= source['B1'],
                                            sourceb2= source['B2'],
                                            ts_list = list(data_bb['B1'].index),
                                            bb_b1   = data_bb['B1'],
                                            bb_b2   = data_bb['B2']),
                                            code    =  """
    //====================================================
    //--------------------------------
    function addvector(a,b){
        return a.map((e,i) => e + b[i]);
    }

    function divvector(a,b){
        return a.map((e,i) => e/b);
    }

    function average2D(array) {
        return divvector(array.reduce((x,y) => addvector(x,y)),array.length);
    }
    //--------------------------------

    const _s     = cb_obj.start-metadata.data['lw'][0]/2;
    const _s_idx = ts_list.findIndex( x => Date.parse(new Date(x)) >= Date.parse(new Date(_s)));
    const _e     = cb_obj.end+metadata.data['lw'][0]/2;
    const _e_idx = ts_list.findIndex( x => Date.parse(new Date(x)) >= Date.parse(new Date(_e)));

    sourceb1.data.data_r = bb_b1[_e_idx];
    sourceb2.data.data_r = bb_b2[_e_idx];

    

    sourceb1.data.data_avg = average2D(bb_b1.slice(_s_idx,_e_idx));
    sourceb2.data.data_avg = average2D(bb_b2.slice(_s_idx,_e_idx));

    sourceb1.change.emit();
    sourceb2.change.emit();
    //====================================================""")


# Efficiecny
#-------------------------------------
for beam,bb_df in zip(beams,[bb_df_b1,bb_df_b2]):
    BOKEH_FIGS[f'Efficiency {beam.name}'] = make_bbbsignature_figure(source,beam,colors =  {'left':'coral','right':'mediumseagreen','avg':'slateblue'})
             
BOKEH_FIGS[f'Efficiency B1B2'] = bkmod.Tabs(tabs=[  bkmod.TabPanel(child=BOKEH_FIGS[f'Efficiency B1'], title="Beam 1"), 
                                                    bkmod.TabPanel(child=BOKEH_FIGS[f'Efficiency B2'], title="Beam 2")])

bk.show(bklay.column(BOKEH_FIGS[f'TOP'],BOKEH_FIGS[f'Efficiency B1B2']))

In [ ]:
pd.DataFrame(source['B2'].data)

In [ ]:
data_I['B1']

In [ ]:
len(data_bb[beam.name][left_ts:right_ts][0])

In [ ]:
len(data_bb[beam.name][left_ts:right_ts].mean())

In [ ]:
dict(time_slider.callback_start_0.args).update({'test':'yesy'})

In [ ]:
data_bb['B2']

In [ ]:
np.min(source['B2'].data['data_l'])

In [ ]:
default_ts